In [1]:
# make_week1_data.py
from datetime import datetime, timedelta
from pathlib import Path
import numpy as np
import pandas as pd
import yfinance as yf
import sys

# -------- CONFIG --------
TICKERS = ["AAPL", "TSLA", "AMZN"]
SECTOR_MAP = {"AAPL": "Tech", "TSLA": "Auto", "AMZN": "Retail"}
N_SESSIONS = 30
LOOKBACK_DAYS = 60
SAVE_DIR = Path("/Users/ngocanh/Documents/Buid/python")
OUTFILE = SAVE_DIR / "Stock_Report_Week1.xlsx"
USE_ADJ_CLOSE = False
# ------------------------

def fetch_prices(tickers, start, end, use_adj=False):
    data = yf.download(
        tickers,
        start=start.strftime("%Y-%m-%d"),
        end=end.strftime("%Y-%m-%d"),
        group_by="ticker",
        auto_adjust=False,
        progress=False
    )
    frames = []
    price_col = "Adj Close" if use_adj else "Close"
    for t in tickers:
        df = (data[t].reset_index()
                        .rename(columns={"Date":"Date", price_col:"Price", "Volume":"Volume"})
                        [["Date","Price","Volume"]]
                        .dropna())
        df["Ticker"] = t
        df["Sector"] = SECTOR_MAP.get(t, "")
        frames.append(df)
    return pd.concat(frames, ignore_index=True)

def build_raw(df, n_sessions):
    raw = (df.sort_values(["Ticker","Date"])
             .groupby("Ticker", group_keys=False).tail(n_sessions)
             .sort_values(["Date","Ticker"])
             .reset_index(drop=True))
    raw["Date"] = pd.to_datetime(raw["Date"]).dt.date
    return raw[["Date","Ticker","Sector","Volume","Price"]]

def build_returns(raw):
    ret = raw.sort_values(["Ticker","Date"]).copy()
    # % change: (P_t - P_{t-1}) / P_{t-1}
    ret["pct_change"] = ret.groupby("Ticker")["Price"].pct_change()
    # log return: ln(P_t / P_{t-1}) — dùng transform để không lệch index
    ret["log_return"] = ret.groupby("Ticker")["Price"].transform(lambda s: np.log(s / s.shift(1)))
    return ret

def save_excel(raw, ret, outfile: Path):
    outfile.parent.mkdir(parents=True, exist_ok=True)
    with pd.ExcelWriter(outfile, engine="openpyxl") as writer:
        raw.to_excel(writer, sheet_name="Raw_Data", index=False)
        ret.to_excel(writer, sheet_name="Returns", index=False)

def main():
    end = datetime.today()
    start = end - timedelta(days=LOOKBACK_DAYS)
    print(f"→ Tải dữ liệu {TICKERS} từ {start.date()} đến {end.date()} ...")
    df_all = fetch_prices(TICKERS, start, end, use_adj=USE_ADJ_CLOSE)

    print("→ Chuẩn hoá Raw_Data (giữ 30 phiên gần nhất / mã)...")
    raw = build_raw(df_all, N_SESSIONS)

    print("→ Tính pct_change & log_return ...")
    ret = build_returns(raw)

    print(f"→ Ghi Excel tới: {OUTFILE}")
    save_excel(raw, ret, OUTFILE)

    print(f"✅ Hoàn tất! Rows: Raw_Data={len(raw)}, Returns={len(ret)}")
    print("   Mở file Excel và làm Pivot/Chart/Conditional theo Week 1.")

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print("❌ Lỗi:", e, file=sys.stderr)
        sys.exit(1)

→ Tải dữ liệu ['AAPL', 'TSLA', 'AMZN'] từ 2025-07-14 đến 2025-09-12 ...
→ Chuẩn hoá Raw_Data (giữ 30 phiên gần nhất / mã)...
→ Tính pct_change & log_return ...
→ Ghi Excel tới: /Users/ngocanh/Documents/Buid/python/Stock_Report_Week1.xlsx
✅ Hoàn tất! Rows: Raw_Data=90, Returns=90
   Mở file Excel và làm Pivot/Chart/Conditional theo Week 1.
